# Capstone Project: Creating a Dataset 

### A description of the problem and a discussion of the background. (15 marks)

For the last couple of months, I have been contemplating claiming my independence as a young adult by moving out of home and renting my own place. However, I have found choosing an area to live a complex and confusing process given the abundance of information and opinions available on Sydney's demographics, and my troubling lack of knowledge about this subject. (maybe the Shire stereotype contains some truth after all...)

To cut through the noise and make a data-driven decision, I want to create and analyse a dataset that will assist in finding the best area for me to move to based on a number of important factors, such as rent prices, crime rates, population density, distance to my work and the proximate lifestyle amenities. 

### A description of the data and how it will be used to solve the problem. (15 marks)

Unfortunately, the data required for this project is dispersed across the web and in a range of different formats. 

The project will use a combination of web scraping and importing downloaded data to centralise the required data, before cleaning and loading it into a dataframe. Some of the datasets include:
- List of relevant LGAs, their geolocations, populations and areas
(Source: Wikipedia) 
- Proximity to work and population density (calculated using Wikipedia data) 
- Average rental price (Source: facs NSW)
- Crime rate (Source: Boscar NSW)
- Most popular venues in each area (Source: FourSquare) 

With the complete dataset, we will conduct some preliminary EDA to understand the data before defining a set of arbitrary preferences and using them to determine the best area to live. 

First, I will need to import the libraries to use for analysis and load the dataset into the notebook 

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
from numpy import nan
from pandas import json_normalize

### Scraping Wikipedia for LGA data

The first step is to scrape Wikipedia to acquire a list of all Local Government Areas (LGAs) in Sydney. The data will be geographically segmented by LGA

In [2]:
# Get HTML data from Wikipedia page (using Response & Beautiful Soup)
url = 'https://en.wikipedia.org/wiki/Local_government_areas_of_New_South_Wales'
response = requests.get(url).content
soup = BeautifulSoup(response, 'html.parser')

In [3]:
# Find the names of each LGA in Sydney & creates a list
lga_table = soup.find('table', {'class':"wikitable sortable"})
lga_table_a = lga_table.findAll('a')

lga_list = []

for lga in lga_table_a: 
    try: 
        lga_list.append(lga.get('title'))
    except: pass
    
# Removes the "(New South Wales)" from LGA names, outputting a list of all the LGA names
lga_list =[i.replace("(New South Wales)",'') for i in lga_list if i]

# Remove Square Kilometre and Square Mile titles
lga_list.remove('Square kilometre')
lga_list.remove('Square mile')

Now we have a clean list of each LGA in Sydney; we can start building our dataframe

In [4]:
# Creates a dataframe and populates it with LGA names 
lga_df = pd.DataFrame(lga_list, columns = ['Local Government Area (LGA)'])
print("Our dataframe contains {} LGAs.".format(lga_df.shape[0]))
lga_df.head()

Our dataframe contains 30 LGAs.


,Local Government Area (LGA)
0,Bayside Council
1,City of Blacktown
2,Municipality of Burwood
3,Camden Council
4,City of Campbelltown


Next, we want to add the area of each LGA from the Wikipedia page into our dataframe. In the HTML code, each cell containing area data has unique attributes which we can use to isolate and extract the relevant data & insert it into our dataframe

In [5]:
# Locating area values & creating a list of every second value (since we don't want area in miles squared)
lga_area = lga_table.find_all("td", attrs = {"style": "text-align:right;"})
lga_area = [i.get_text().rstrip() for i in lga_area][::2]

# Insert as a new column into the dataframe 
lga_df['Area (Km squared)'] = lga_area
lga_df['Area (Km squared)'] = lga_df['Area (Km squared)'].astype(float)
lga_df.head()

,Local Government Area (LGA),Area (Km squared)
0,Bayside Council,50.0
1,City of Blacktown,240.0
2,Municipality of Burwood,7.1
3,Camden Council,201.0
4,City of Campbelltown,312.0


### Finding, cleaning and adding data on rent prices

This next section is for finding, clean & incorporating data on the median rent prices in each LGA

In [6]:
# This loads in a CSV file on rent prices in each LGA and drops irrelevant columns
rent_data = pd.read_csv("/Users/alexdelacey/Desktop/Capstone_data/Rent_data_LGA.csv", encoding='latin-1')
rent_data.head()

,ï»¿Greater Metropolitan Region (GMR),Greater Sydney,Rings,Local Government Area (LGA),Dwelling Types,Number of Bedrooms,Median Weekly Rent for New Bonds\n$,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Total,Greater Sydney,Total,Total,Total,Total,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,Greater Sydney,Total,Total,Total,Bedsitter,350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total,Greater Sydney,Total,Total,Total,1 Bedroom,440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Total,Greater Sydney,Total,Total,Total,Not Specified,430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Total,Greater Sydney,Total,Total,Total,2 Bedrooms,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Removing more irrelevant columns 
rent_data = rent_data.iloc[:,:7]
rent_data = rent_data.dropna()
rent_data.head()

,ï»¿Greater Metropolitan Region (GMR),Greater Sydney,Rings,Local Government Area (LGA),Dwelling Types,Number of Bedrooms,Median Weekly Rent for New Bonds\n$
0,Total,Greater Sydney,Total,Total,Total,Total,500
1,Total,Greater Sydney,Total,Total,Total,Bedsitter,350
2,Total,Greater Sydney,Total,Total,Total,1 Bedroom,440
3,Total,Greater Sydney,Total,Total,Total,Not Specified,430
4,Total,Greater Sydney,Total,Total,Total,2 Bedrooms,500


In [8]:
# Iscolates data for all dwelling types & all number of bedrooms
rent_data_sorted = rent_data[rent_data['Number of Bedrooms'] == 'Total']
rent_data_sorted = rent_data_sorted[rent_data['Dwelling Types'] == 'Total']

<ipython-input-8-34f6f54ef52f>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rent_data_sorted = rent_data_sorted[rent_data['Dwelling Types'] == 'Total']


In [9]:
# drops irrrelevant columns & four LGAs that are outside the Sydney region 
rent_data_sorted = rent_data_sorted.iloc[:,[3,6]].reset_index()
rent_data_sorted = rent_data_sorted.drop(axis = 0, index=[0,3,9,13,33]).reset_index()

# Adds Median Weekly Rent as a column in our original LGA dataframe
lga_df['Median Weekly Rent ($)'] = rent_data_sorted['Median Weekly Rent for New Bonds\n$'].astype(int)
lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($)
0,Bayside Council,50.0,530
1,City of Blacktown,240.0,420
2,Municipality of Burwood,7.1,500
3,Camden Council,201.0,510
4,City of Campbelltown,312.0,400


### Scraping Wikipedia for geolocation data

The dataframe now includes rent prices. Next, we want the geographical coordinates for each LGA as this will be useful for calculating distances and visualising our data later. The next section of code will scrape geolocations off the Wikipedia page for each LGA in Sydney. 

In [10]:
# An empty list for latitude & longitude data
lga_lat = []
lga_long = []

# First half of the URL for each LGA in our list 
wikipedia_domain = "https://en.wikipedia.org/wiki/" 

# Function takes in a web address, converts it to a BeautifulSoup object, extracts geolocation and puts it 
# into the lists for latitude and longitude 
def wiki_scrape_geoloc(web_address): 
    request = requests.get(web_address)
    bs = BeautifulSoup(request.text)
    # Finds geolocations from table 
    wiki_table = bs.find(attrs = {'class':'infobox vcard'})
    latitude = wiki_table.find(attrs = {'class':'latitude'}).get_text()
    longitude = wiki_table.find(attrs = {'class':'longitude'}).get_text()
    lga_lat.append(latitude)
    lga_long.append(longitude)  
    
# Iterating over each LGA name in our dataframe and trying to scrape geolocation from Wikipedia
success_count, fail_count = 0, 0
for lga in lga_df['Local Government Area (LGA)']: 
    # Tries the wiki_scrape_geoloc function on wiki address + lga name with underscores replacing spaces
    try: 
        web_address = (wikipedia_domain + lga.replace(' ', '_'))
        wiki_scrape_geoloc(web_address)
        success_count += 1 
    except: 
        # Tries the wiki_scrape_geoloc function on wiki address + lga name with underscores replacing spaces & 
        # bracketed New South Wales added to the end 
        try: 
            web_address = (wikipedia_domain + lga.replace(' ', '_') + "(New_South_Wales)")
            wiki_scrape_geoloc(web_address)
            success_count += 1 
        except: 
            # Appends rows with unsuccessful scrape attempts with Null (NaN) values 
            lga_lat.append(nan)
            lga_long.append(nan)
            web_address = (wikipedia_domain + lga.replace(' ', '_'))
            fail_count += 1 
            print("!Co-ordinates not on Wiki table for", lga, web_address)
            
print("Function successfully scraped geo-coordinates from {} pages, failed on {} pages".format(success_count, fail_count))

# Adding coordinates into main dataframe 
lga_df['Latitude'], lga_df['Longitude'] = lga_lat, lga_long

!Co-ordinates not on Wiki table for Bayside Council https://en.wikipedia.org/wiki/Bayside_Council
!Co-ordinates not on Wiki table for City of Canterbury-Bankstown https://en.wikipedia.org/wiki/City_of_Canterbury-Bankstown
!Co-ordinates not on Wiki table for Cumberland Council  https://en.wikipedia.org/wiki/Cumberland_Council_
!Co-ordinates not on Wiki table for Inner West Council https://en.wikipedia.org/wiki/Inner_West_Council
!Co-ordinates not on Wiki table for Municipality of Strathfield https://en.wikipedia.org/wiki/Municipality_of_Strathfield
Function successfully scraped geo-coordinates from 25 pages, failed on 5 pages


25 of the 30 geolocation datapoints were scraped successfully from Wikipedia in an automated manner. The LGAs for which our scraping failed didn't have geocoordinates listed on their Wikipedia page, explaining why the code failed for 5 LGAs. Let's add the geolocations we did scrape into our dataframe and see how it looks before deciding how to deal with missing values.


The geolocation data will need to be cleaned before it is functional. The cleaning will include the following: 

1. Our geographical coordinates must be converted from a DMS to a decimal format for later use as an input into the Foursquare API 
2. Missing geolocation data needs to be added to the dataframe 
3. We need to remove the degree symbol, apostrophe and letter from each value 
4. Our latitude values need to be negative; as Sydney is below the equator & our location searches will otherwise end up in the Pacific Ocean...

The first code block will convert coordinates from degrees to decimal format.

In [11]:
# Function takes geolocation in degrees format and outputs goecoordinates in a decimal format 
def convert_to_decimal(degrees):
    deg_i = degrees.find("°")
    characteristic, mantissa = degrees[:deg_i], degrees[deg_i+1:deg_i+3]
    converted = str(round(int(mantissa) * 100/60))
    if len(converted) == 1: 
        return characteristic + '.0' + converted + ' ' + degrees[-1]
    else: 
        return characteristic + '.' + converted + ' ' + degrees[-1]
    
# Converting Latitude values to decimal & replacing in dataframe
for index, val in enumerate(lga_df['Latitude']): 
    try: lga_df.at[index, 'Latitude'] = convert_to_decimal(val)
    except: pass

# Converting Longitude values to decimal & replacing in dataframe
for index, val in enumerate(lga_df['Longitude']): 
    try: lga_df.at[index, 'Longitude'] = convert_to_decimal(val)
    except: pass

Adding in missing geolocation data: 

In [12]:
# Dictionary for each LGA with missing geolocation
manual_coords = {"Bayside Council":("33.94 S", "151.17 E"), "City of Canterbury-Bankstown":("33.91 S","151.03 E"), 
 "Cumberland Council":("33.83 S", "150.96 E"), "Inner West Council":("33.85 S", "151.13 E"),
 "Municipality of Strathfield": ("33.88 S", "151.08 E")
} 

# Finding index of each row with a null value and creating a list
null_coords = lga_df['Latitude'].isnull()
null_index = [i for i in null_coords.index if null_coords[i] == True]

# Inserting values into position in dataframe 
for i in range(len(null_index)):
    lga_df.at[null_index[i], 'Latitude'] = list(manual_coords.values())[i][0]
    lga_df.at[null_index[i], 'Longitude'] = list(manual_coords.values())[i][1]

# Clean the Coordinate format for Ku-ring-gai Council manually 
lga_df.at[13, 'Longitude'] = "151.15 E"
lga_df.at[13, 'Latitude'] = "33.74 S"

Converting latitude values to negative and removing letters from our geolocation values

In [13]:
# Remove letters and add a negative sign for latitude 
for i in range(len(lga_df)): 
    lga_df.at[i, 'Latitude'] = '-' + lga_df.at[i, 'Latitude'][:-2]

# Remove letters from latitude
for i in range(len(lga_df)): 
    lga_df.at[i, 'Longitude'] = lga_df.at[i, 'Longitude'][:-2]
    
lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($),Latitude,Longitude
0,Bayside Council,50.0,530,-33.94,151.17
1,City of Blacktown,240.0,420,-33.77,150.92
2,Municipality of Burwood,7.1,500,-33.88,151.10
3,Camden Council,201.0,510,-34.05,150.70
4,City of Campbelltown,312.0,400,-34.07,150.82


### Using geolocation data to calculate the distance between each LGA & my work and CBD


I can calculate the distance between my work and each LGA using each LGA's geolocation, along with the distance of each LGA from the CBD. I will start by finding the geolocation of both the CBD and my work suburb on google: 

In [14]:
# Hard coding values for Redfern & the CBD
redfern_lat = -33.8930
redfern_long = 151.2054
CBD_lat = -33.8708
CBD_long = 151.2073

We will need a mathematical function to calculate the distance in kms between two geolocations

In [15]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

We can now use this function to calculate the distance between each LGA and Redfern

In [16]:
# Empty list for each distance 
distance_work_km = []
distance_CBD_km = []

# Finds latitude, longtitude for each LGA and runs it through our haversine_distance function 
for i in range(len(lga_df)):
    lat1 = float(lga_df.at[i, 'Latitude']) 
    long1 = float(lga_df.at[i, 'Longitude'])
    distance_work_km.append(haversine_distance(lat1, long1, redfern_lat, redfern_long))
    distance_CBD_km.append(haversine_distance(lat1, long1, CBD_lat, CBD_long))
    
# Updating the dataframe with new distance from work values: 
lga_df["Distance from Work (Kms)"] = distance_work_km
lga_df["Distance from CBD (Kms)"] = distance_CBD_km
lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($),Latitude,Longitude,Distance from Work (Kms),Distance from CBD (Kms)
0,Bayside Council,50.0,530,-33.94,151.17,6.16,8.43
1,City of Blacktown,240.0,420,-33.77,150.92,29.70,28.81
2,Municipality of Burwood,7.1,500,-33.88,151.10,9.84,9.96
3,Camden Council,201.0,510,-34.05,150.70,49.77,50.85
4,City of Campbelltown,312.0,400,-34.07,150.82,40.62,42.03


### Finding and cleaning crime data

Next, I will load in and clean crime data before adding this to the dataset. I will need to isolate data from just 2019 and separating Sydney LGAs from non-Sydney LGAs

In [17]:
# Loading each Excel sheet into data a seperate pandas dataframe 
xls = pd.ExcelFile("/Users/alexdelacey/Desktop/Capstone_data/LgaRankings_27_Offences (2).xlsx")

assault_df = pd.read_excel(xls, 'Assault - non-domestic violence', skiprows = 5, skipfooter = 6)
robbery_df = pd.read_excel(xls, 'Robbery', skiprows = 5, skipfooter = 6)
break_enter_df = pd.read_excel(xls, 'Break and enter dwelling', skiprows = 5, skipfooter = 6)

In [18]:
# Selecting the correct sheet for assault, robbery & break and enter tables
assault_df = assault_df[['Local Government Area', 'Rate per 100,000 population.4']]
robbery_df = robbery_df[['Local Government Area', 'Rate per 100,000 population.4']]
break_enter_df = robbery_df[['Local Government Area', 'Rate per 100,000 population.4']]

# Loading in a list of lga names in proper format 
relevent_lgas = pd.read_csv('/Users/alexdelacey/Desktop/Capstone_data/LGA_Names.csv')
relevent_lgas = [i.lower() for i in relevent_lgas['0']]

In [19]:
# Iterating over each sheet, selecting Sydney only LGAs 
for i in assault_df['Local Government Area']: 
    if i.lower() not in relevent_lgas: 
        x = (assault_df[assault_df['Local Government Area']== i].index)
        assault_df = assault_df.drop(x).reset_index(drop = True)
assault_df.reset_index(drop = True)

for i in robbery_df['Local Government Area']: 
    if i.lower() not in relevent_lgas:
        x = (robbery_df[robbery_df['Local Government Area']== i].index)
        robbery_df = robbery_df.drop(x)
robbery_df = robbery_df.reset_index(drop = True)

for i in break_enter_df['Local Government Area']:
    if i.lower() not in relevent_lgas:
        x = (break_enter_df[break_enter_df['Local Government Area']== i].index)
        break_enter_df = break_enter_df.drop(x)
break_enter_df = break_enter_df.reset_index(drop = True)  

In [20]:
# Adding columns for assault, robbery & break and enter into the main dataframe
lga_df['Assault per 100,000'] = assault_df['Rate per 100,000 population.4']
lga_df['Robberies per 100,000'] = robbery_df['Rate per 100,000 population.4']
lga_df['Break and Enter per 100,000'] = break_enter_df['Rate per 100,000 population.4']

lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($),Latitude,Longitude,Distance from Work (Kms),Distance from CBD (Kms),"Assault per 100,000","Robberies per 100,000","Break and Enter per 100,000"
0,Bayside Council,50.0,530,-33.94,151.17,6.16,8.43,294.188,29.8203,29.8203
1,City of Blacktown,240.0,420,-33.77,150.92,29.70,28.81,467.624,61.9315,61.9315
2,Municipality of Burwood,7.1,500,-33.88,151.10,9.84,9.96,263.25,52.6501,52.6501
3,Camden Council,201.0,510,-34.05,150.70,49.77,50.85,216.655,8.4963,8.4963
4,City of Campbelltown,312.0,400,-34.07,150.82,40.62,42.03,398.48,48.1744,48.1744


### Using the Foursquare API to find popular venues in each LGA

I want to identify the 3 most popular venue types for each LGA

In [40]:
# Variables for all of the URL inputs 
CLIENT_ID = 'AMBG0WJK3FQ2NBDQNCXCSYOOKKLGDZG21H4DIO0OJKIZYJ3J' 
CLIENT_SECRET = 'GVU3SMQJBDEIV4RIMIUYUKDTDM0WTPKXOWCLYZDGDVYOVECU' 
VERSION = '20180605'
LIMIT = 100 
Radius = 10000
lat = lga_df.at[0, 'Latitude']
lon = lga_df.at[0, 'Longitude']

{'tags': ['hide-input']}

In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&radius={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, Radius, VERSION)
results = requests.get(url).json()

In [23]:
# function extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yan Restaurant,Asian Restaurant,-33.931950,151.150870
1,Pullman Sydney Airport,Hotel,-33.927565,151.186850
2,Mascot Kebabs & Pides,Middle Eastern Restaurant,-33.930711,151.194350
3,Bourke Street Bakery,Bakery,-33.920952,151.193621
4,Brothers Kebabs,Kebab Restaurant,-33.945028,151.141616


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
S_venues = getNearbyVenues(names=lga_df['Local Government Area (LGA)'],
                                   latitudes= lga_df['Latitude'],
                                   longitudes=lga_df['Longitude']
                                  )

Bayside Council
City of Blacktown
Municipality of Burwood
Camden Council 
City of Campbelltown 
City of Canada Bay
City of Canterbury-Bankstown
Cumberland Council 
City of Fairfield
Georges River Council
Hornsby Shire
Municipality of Hunter's Hill
Inner West Council
Ku-ring-gai Council
Lane Cove Council
City of Liverpool 
Mosman Council
North Sydney Council
Northern Beaches Council
City of Parramatta
City of Penrith
City of Randwick
City of Ryde
Municipality of Strathfield
Sutherland Shire
City of Sydney
The Hills Shire
Waverley Municipal Council
City of Willoughby
Municipality of Woollahra


In [27]:
print(S_venues.shape)
S_venues.head()

(2956, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bayside Council,-33.94,151.17,Yan Restaurant,-33.931950,151.150870,Asian Restaurant
1,Bayside Council,-33.94,151.17,Pullman Sydney Airport,-33.927565,151.186850,Hotel
2,Bayside Council,-33.94,151.17,Mascot Kebabs & Pides,-33.930711,151.194350,Middle Eastern Restaurant
3,Bayside Council,-33.94,151.17,Bourke Street Bakery,-33.920952,151.193621,Bakery
4,Bayside Council,-33.94,151.17,Brothers Kebabs,-33.945028,151.141616,Kebab Restaurant


In [28]:
S_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayside Council,100,100,100,100,100,100
Camden Council,66,66,66,66,66,66
City of Blacktown,100,100,100,100,100,100
City of Campbelltown,90,90,90,90,90,90
City of Canada Bay,100,100,100,100,100,100
City of Canterbury-Bankstown,100,100,100,100,100,100
City of Fairfield,100,100,100,100,100,100
City of Liverpool,100,100,100,100,100,100
City of Parramatta,100,100,100,100,100,100


In [29]:
# Let's find out how many unique categories of venues is there from all the returned venues

print('There are {} uniques categories.'.format(len(S_venues['Venue Category'].unique())))

There are 203 uniques categories.


In [30]:
# Let's analyze each neighborhood
# One Hot encoding

# one hot encoding
Syd_onehot = pd.get_dummies(S_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Syd_onehot['Neighborhood'] = S_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Syd_onehot.columns[-1]] + list(Syd_onehot.columns[:-1])
Syd_onehot = Syd_onehot[fixed_columns]

Syd_onehot.head()

,Neighborhood,Afghan Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Bayside Council,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bayside Council,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bayside Council,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayside Council,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Bayside Council,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
Syd_groupOHE = Syd_onehot.groupby("Neighborhood").mean().reset_index()
Syd_groupOHE.head()

,Neighborhood,Afghan Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Bayside Council,0.00,0.010000,0.0,0.0,0.01,0.00,0.000000,0.01,0.060000,...,0.02,0.0,0.010000,0.000000,0.0,0.0,0.01,0.01,0.01,0.00
1,Camden Council,0.00,0.000000,0.0,0.0,0.00,0.00,0.015152,0.00,0.000000,...,0.00,0.0,0.015152,0.015152,0.0,0.0,0.00,0.00,0.00,0.00
2,City of Blacktown,0.01,0.000000,0.0,0.0,0.00,0.01,0.030000,0.00,0.010000,...,0.00,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.00,0.01
3,City of Campbelltown,0.00,0.011111,0.0,0.0,0.00,0.00,0.022222,0.00,0.011111,...,0.00,0.0,0.033333,0.011111,0.0,0.0,0.00,0.00,0.00,0.00
4,City of Canada Bay,0.00,0.010000,0.0,0.0,0.00,0.00,0.000000,0.00,0.020000,...,0.00,0.0,0.000000,0.000000,0.0,0.0,0.01,0.00,0.00,0.00


In [32]:
# Sort data in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
# Creating new data frame with top 10 venues
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Syd_groupOHE['Neighborhood']

for ind in np.arange(Syd_groupOHE.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Syd_groupOHE.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bayside Council,Café,Ice Cream Shop,Bar
1,Camden Council,Fast Food Restaurant,Café,Supermarket
2,City of Blacktown,Park,Burger Joint,Café
3,City of Campbelltown,Fast Food Restaurant,Grocery Store,Supermarket
4,City of Canada Bay,Café,Park,Bar


In [34]:
# Now, lets add these columns into our main dataframe: 

lga_df = pd.concat([lga_df, neighborhoods_venues_sorted], axis=1)
lga_df = lga_df.drop(columns = ['Neighborhood'])
lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($),Latitude,Longitude,Distance from Work (Kms),Distance from CBD (Kms),"Assault per 100,000","Robberies per 100,000","Break and Enter per 100,000",1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bayside Council,50.0,530,-33.94,151.17,6.16,8.43,294.188,29.8203,29.8203,Café,Ice Cream Shop,Bar
1,City of Blacktown,240.0,420,-33.77,150.92,29.70,28.81,467.624,61.9315,61.9315,Fast Food Restaurant,Café,Supermarket
2,Municipality of Burwood,7.1,500,-33.88,151.10,9.84,9.96,263.25,52.6501,52.6501,Park,Burger Joint,Café
3,Camden Council,201.0,510,-34.05,150.70,49.77,50.85,216.655,8.4963,8.4963,Fast Food Restaurant,Grocery Store,Supermarket
4,City of Campbelltown,312.0,400,-34.07,150.82,40.62,42.03,398.48,48.1744,48.1744,Café,Park,Bar


### Adding data on Age 

Adding in age distribution data to understand the different age distributions accross Sydney

In [35]:
# Loads in the data, skipping redundant rows: 
xls = pd.ExcelFile("/Users/alexdelacey/Desktop/Capstone_data/32350ds0003_2019.xls")
age_df = pd.read_excel(xls, 'Table 3', skiprows = [0,1,2,3,4,5])
print("Dataframe shape:", age_df.shape)

Dataframe shape: (553, 23)


In [36]:
# Cleaning the dataset: 

# Grouping correct column headers together into one row, then adding as column headers.
age_ranges = [age_group for age_group in age_df.iloc[0, 4:]] 
columns = [header for header in age_df.iloc[1, :4]]

if age_df.iloc[1,4] == 'no.':
    age_df.columns = columns + age_ranges
    # Dropping empty rows 
    age_df = age_df.drop([0,1,2, 547, 548, 549, 550, 551, 552],).reset_index(drop = True)
    
# Removing all rows outside of NSW
age_df = age_df[age_df['S/T name'] == 'New South Wales']

# Removing rows not in Sydney
def name_clean(value):  
    return value.split('(')[0][:-1].lower() 
age_df['LGA name'] = age_df['LGA name'].apply(name_clean)

for i, lga in enumerate(age_df['LGA name']): 
    if lga not in relevent_lgas: 
        age_df = age_df.drop(i)
        
age_df_1 = age_df.iloc[:,4:]
age_df_1 = age_df_1.reset_index(drop = True) 

for col in age_df_1: 
    values = (age_df_1[col])
    lga_df[col] = values

Now that we have population data in the dataframe, lets create a measure for population density and another one for median age

In [37]:
# Finding the median from a grouped frequency distribution

# Create a function that finds median age group
def find_median(row):
    
    start_index = lga_df.columns.get_loc('0-4')
    end_index = lga_df.columns.get_loc('85 and over')
    age_range_columns = lga_df.columns[start_index:end_index]
    
    total_persons_index = lga_df.columns.get_loc('Total Persons')
    total_persons = lga_df.iloc[row, total_persons_index]
    midpoint = total_persons/2
    
    pop_per_group = [i for i in lga_df.iloc[row]][start_index:end_index]
    
    cumulative_sum = []
    index = 0
    for pop in pop_per_group:
        if sum(cumulative_sum + [pop]) < midpoint: 
            cumulative_sum.append(pop)
            index += 1 
    
    return age_range_columns[index]

median = [find_median(i) for i in range(len(lga_df))]
lga_df['Median age group'] = median

# Adding measure for population density into the main dataframe
lga_df["Population Density (People per square Km)"] = (lga_df["Total Persons"] / lga_df["Area (Km squared)"]).astype(int)
lga_df.head()

,Local Government Area (LGA),Area (Km squared),Median Weekly Rent ($),Latitude,Longitude,Distance from Work (Kms),Distance from CBD (Kms),"Assault per 100,000","Robberies per 100,000","Break and Enter per 100,000",...,55–59,60–64,65–69,70–74,75–79,80–84,85 and over,Total Persons,Median age group,Population Density (People per square Km)
0,Bayside Council,50.0,530,-33.94,151.17,6.16,8.43,294.188,29.8203,29.8203,...,9371,7904,7148,5999,4535,3581,3901,178396,35–39,3567
1,City of Blacktown,240.0,420,-33.77,150.92,29.70,28.81,467.624,61.9315,61.9315,...,19635,16781,13706,10779,6663,4487,3853,374451,35–39,1560
2,Municipality of Burwood,7.1,500,-33.88,151.10,9.84,9.96,263.25,52.6501,52.6501,...,2189,1850,1473,1303,976,872,1136,40612,35–39,5720
3,Camden Council,201.0,510,-34.05,150.70,49.77,50.85,216.655,8.4963,8.4963,...,4937,3987,3278,2682,1625,1032,962,101437,35–39,504
4,City of Campbelltown,312.0,400,-34.07,150.82,40.62,42.03,398.48,48.1744,48.1744,...,10046,8944,7628,5972,3544,2171,1750,170943,35–39,547


We now have a dataset with all the information required for me to make a decision on where I want to live. Lets convert it to a CSV file so it can be exported

In [38]:
lga_df.to_csv(r'/Users/alexdelacey/Desktop/Capstone_data/Suburb_dataset.csv', index = False, header = True)